In [82]:
!rm taxi.sqlite

rm: cannot remove 'taxi.sqlite': Device or resource busy


In [83]:
import pandas as pd

In [84]:
taxi_data = "taxi_data_12-31-2016.csv"
taxi_data_df = pd.read_csv(taxi_data) 

In [85]:
taxi_data_df.head(3)

,trip_id,trip_start_timestamp,trip_end_timestamp,fare,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude
0,32761dfbd17e17c7594369a27b47d01e42b9d81c,12/31/2016 19:30,12/31/2016 19:30,0.0,NaN,NaN,NaN,NaN
1,79429ce2f9829733a2b1b6f69ae5b142328657a7,12/31/2016 9:15,12/31/2016 9:15,5.0,NaN,NaN,NaN,NaN
2,7e0faa899f6a26956b36869b9b1611eada9fcc89,12/31/2016 14:45,12/31/2016 14:45,5.0,NaN,NaN,NaN,NaN


In [86]:
clean_taxi_df = taxi_data_df.dropna(how='any')

In [87]:
clean_taxi_df.head(10)

,trip_id,trip_start_timestamp,trip_end_timestamp,fare,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude
11,02847db485a2d331ad991e73aadd1e7930ebde97,12/31/2016 1:30,12/31/2016 1:45,6.25,41.901207,-87.676356,41.901207,-87.676356
12,03c31816c81af2c86c9289c22359c22507cfaffe,12/31/2016 22:15,12/31/2016 22:30,6.75,41.922761,-87.699155,41.922761,-87.699155
13,0417a35f9db99f8828887fdd42953d6c85b2f105,12/31/2016 19:45,12/31/2016 20:00,4.75,41.965812,-87.655879,41.965812,-87.655879
15,07a800a010437e5718c8c46f6d5bc6d4ebadb3c3,12/31/2016 23:15,12/31/2016 23:30,8.00,41.885281,-87.657233,41.893216,-87.637844
16,0a4bf27162d338893ad9cd33048f4838d4dcbf2c,12/31/2016 0:15,12/31/2016 0:30,8.25,41.899602,-87.633308,41.899602,-87.633308
17,0b313871bb5e22434be939777b4773d4bc20ec32,12/31/2016 17:00,12/31/2016 17:15,20.00,41.899602,-87.633308,41.944227,-87.655998
18,0e16e461b21f26b0ac3583fa5d5fb286b3e4a8a3,12/31/2016 14:15,12/31/2016 14:45,15.25,41.922686,-87.649489,41.874005,-87.663518
19,0efe698106a8086fab8d3c59fc97f6441a0e0bd6,12/31/2016 18:15,12/31/2016 18:30,4.25,41.965812,-87.655879,41.965812,-87.655879
20,1198c3af81853a68408a89a6a9a04dd6445d42c3,12/31/2016 14:45,12/31/2016 14:45,6.75,41.922686,-87.649489,41.944227,-87.655998
22,16b4eb24958b8987f245b013bdcca17fe36a2fc4,12/31/2016 10:15,12/31/2016 10:30,7.25,41.878866,-87.625192,41.874005,-87.663518


In [88]:
clean_taxi_df.count()

trip_id                 28561
trip_start_timestamp    28561
trip_end_timestamp      28561
fare                    28561
pickup_latitude         28561
pickup_longitude        28561
dropoff_latitude        28561
dropoff_longitude       28561
dtype: int64

In [89]:
help(clean_taxi_df.to_sql)

Help on method to_sql in module pandas.core.generic:

to_sql(name, con, schema=None, if_exists='fail', index=True, index_label=None, chunksize=None, dtype=None) method of pandas.core.frame.DataFrame instance
    Write records stored in a DataFrame to a SQL database.
    
    Databases supported by SQLAlchemy [1]_ are supported. Tables can be
    newly created, appended to, or overwritten.
    
    Parameters
    ----------
    name : string
        Name of SQL table.
    con : sqlalchemy.engine.Engine or sqlite3.Connection
        Using SQLAlchemy makes it possible to use any DB supported by that
        library. Legacy support is provided for sqlite3.Connection objects.
    schema : string, optional
        Specify the schema (if database flavor supports this). If None, use
        default schema.
    if_exists : {'fail', 'replace', 'append'}, default 'fail'
        How to behave if the table already exists.
    
        * fail: Raise a ValueError.
        * replace: Drop the table be

In [90]:
# Import SQLAlchemy `automap` and other dependencies
from sqlalchemy import create_engine, inspect

In [91]:
# Create the connection engine
engine = create_engine("sqlite:///taxi.sqlite")
conn = engine.connect()

In [92]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [93]:
from sqlalchemy import Column, Integer, String, Float
    
class Metadata(Base):
    __tablename__ = "taxi_data"
    trip_id = Column(String, primary_key=True)
    trip_start_timestamp = Column(String)
    trip_end_timestamp = Column(String)
    pickup_latitude = Column(Float)
    pickup_longitude = Column(Float)
    dropoff_latitude = Column(Float)
    dropoff_longitude = Column(Float)
    fare = Column(Float)

In [94]:
Base.metadata.create_all(conn)

In [95]:
clean_taxi_df.to_sql(Metadata.__tablename__, conn, index=False, if_exists="replace")

In [96]:
import pymysql, os
pymysql.install_as_MySQLdb()

In [97]:
engine = create_engine("postgres://{postgress_url}")
conn = engine.connect()

In [98]:
Base.metadata.drop_all(conn)
Base.metadata.create_all(conn)

In [99]:
clean_taxi_df.to_sql(Metadata.__tablename__, conn, index=False, if_exists="append")